In [1]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from rasterstats import zonal_stats
import math
import warnings
warnings.filterwarnings("ignore")

## Load shapefile

In [3]:
shp_attr = r"D:\MPSeDC\Kharif\RS_Yield_Kharif_2024_Soybean.shp"   # shapefile with attribute to rasterize
shp_zone = r"D:\MPSeDC\UTM\Village_UTM.shp"  # shapefile for zonal statistics (FID field)

gdf_attr = gpd.read_file(shp_attr)
gdf_zone = gpd.read_file(shp_zone)

# Make sure both are in the same CRS
if gdf_zone.crs != gdf_attr.crs:
    gdf_attr = gdf_attr.to_crs(gdf_zone.crs)
crs = gdf_attr.crs
crs 

<Projected CRS: EPSG:32643>
Name: WGS 84 / UTM zone 43N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 72°E and 78°E, northern hemisphere between equator and 84°N, onshore and offshore. China. India. Kazakhstan. Kyrgyzstan. Maldives. Pakistan. Russian Federation. Tajikistan.
- bounds: (72.0, 0.0, 78.0, 84.0)
Coordinate Operation:
- name: UTM zone 43N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
gdf = gdf_zone.head()
print(gdf)

  code class sub_class       vil_nm_h        vil_nm_e lgd_gp_nm  \
0   01    01        13     डाँग सरकार      Dangsarkar   Adupura   
1   01    01        13   आहुखाना कलाँ  Ahukhana kalan      None   
2   01    01        13  आहुखाना खुर्द  Ahukhana khurd      None   
3   01    01        13         ओहदपुर         Ohadpur      None   
4   01    01        13      कल्यानबाग       Kalyanbag      None   

           teh_nm       b_nm  dist_nm   div_nm  ... pc_cd state_cd  \
0  Gwalior Gramin      Morar  Gwalior  Gwalior  ...   003       23   
1   Gwalior(Gird)  Ghatigaon  Gwalior  Gwalior  ...   003       23   
2   Gwalior(Gird)  Ghatigaon  Gwalior  Gwalior  ...   003       23   
3     City Center  Ghatigaon  Gwalior  Gwalior  ...   003       23   
4   Gwalior(Gird)  Ghatigaon  Gwalior  Gwalior  ...   003       23   

    area_hecta lgd_gp_cd teh_cd  vil_cd          bhu_cd ccode_11   lgdcd  \
0  2936.526106    139384  07395  453859  03090300036218   453859  453859   
1   268.513648      None

## Rasterize shapefile

In [5]:
# Rasterize shapefile A
resolution = 10  # meters per pixel

# Use the correct variable name
bounds = gdf_attr.total_bounds
width = int((bounds[2] - bounds[0]) / resolution)
height = int((bounds[3] - bounds[1]) / resolution)

from rasterio.transform import from_bounds
transform = from_bounds(*bounds, width, height)

In [6]:
# transform = from_origin(minx, maxy, resolution, resolution)

field = 'Soy_Yield'   # put your attribute column here

shapes = [(geom, value) for geom, value in zip(gdf_attr.geometry, gdf_attr[field])]

raster = rasterize(
    shapes=shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,
    dtype="int16"
)

out_tif = r"D:\MPSeDC\Vijay_Project\Polygon_To_Raster\RS_Yield_Kharif_2024_Soybean_1.tif"
with rasterio.open(
    out_tif,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=1,
    dtype=raster.dtype,
    crs=gdf_attr.crs,
    transform=transform,
) as dst:
    dst.write(raster, 1)

# print(f"Raster saved at {out_tif}")

## Zonal Statistics with shapefile
##### 35 minute 

In [7]:

stats = zonal_stats(
    vectors=gdf_zone,
    raster=out_tif,
    stats=["mean"],  # choose statistics
    geojson_out=True
)

# Convert to GeoDataFrame
gdf_stats = gpd.GeoDataFrame.from_features(stats)


# Save results

# gdf_stats.to_file(r"D:\MPSeDC\vijay\Polygon_To_Raster\RS_Yield_Kharif_2024_Soybean_Zonal.dbf")

print("Zonal statistics saved as shapefile!")

Zonal statistics saved as shapefile!
